In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

%load_ext autoreload
%autoreload 2

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
texas = catalog.load('texas.charges')()

WARNING  Loading partitioned dataset as a whole, this may cause memory issues.                                           __init__.py:85


In [ ]:
# Optimised load = RAM use for CSV, Parquet, and csv.gz
texas.drop(columns=['modifier_3', 'modifier_4']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508660708 entries, 0 to 508660707
Data columns (total 11 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   record_id              int64   
 1   revenue_code           category
 2   hcpcs_qualifier        category
 3   hcpcs_procedure_code   category
 4   modifier_1             category
 5   modifier_2             category
 6   unit_measurement_code  category
 7   units_of_service       Int32   
 8   unit_rate              float32 
 9   chrgs_line_item        float32 
 10  chrgs_non_cov          float32 
dtypes: Int32(1), category(6), float32(3), int64(1)
memory usage: 16.6 GB


In [ ]:
# 3GB
texas.drop(columns=['modifier_3', 'modifier_4']).to_parquet("../../data/texas_tmp.pq")

In [ ]:
import pandas as pd

%timeit pd.read_parquet("../../data/texas_tmp.pq")

24 s ± 890 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import pyarrow as pa

texas_pa = pa.Table.from_pandas(texas.drop(columns=['modifier_3', 'modifier_4']))

In [ ]:
import pyarrow.csv as csv

with pa.CompressedOutputStream("../../data/texas_tmp.csv.gz", "gzip") as out:
    csv.write_csv(texas_pa, out)

In [ ]:
# $ gzip -dk texas_tmp.csv.gz

In [ ]:
import pandas as pd
texas.drop(columns=['modifier_3', 'modifier_4']).dtypes


record_id                   int64
revenue_code             category
hcpcs_qualifier          category
hcpcs_procedure_code     category
modifier_1               category
modifier_2               category
unit_measurement_code    category
units_of_service            Int32
unit_rate                 float32
chrgs_line_item           float32
chrgs_non_cov             float32
dtype: object


In [ ]:
texas_csv = pd.read_csv(
    "../../data/texas_tmp.csv.gz",
    dtype={
        "revenue_code": "category",
        "hcpcs_qualifier": "category",
        "hcpcs_procedure_code": "category",
        "modifier_1": "category",
        "modifier_2": "category",
        "unit_measurement_code": "category",
        "units_of_service": "Int32",
        "unit_rate": "float32",
        "chrgs_line_item": "float32",
        "chrgs_non_cov": "float32",
    },
)


In [ ]:
texas_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508660708 entries, 0 to 508660707
Data columns (total 11 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   record_id              int64   
 1   revenue_code           category
 2   hcpcs_qualifier        category
 3   hcpcs_procedure_code   category
 4   modifier_1             category
 5   modifier_2             category
 6   unit_measurement_code  category
 7   units_of_service       Int32   
 8   unit_rate              float32 
 9   chrgs_line_item        float32 
 10  chrgs_non_cov          float32 
dtypes: Int32(1), category(6), float32(3), int64(1)
memory usage: 16.6 GB


In [ ]:
p = catalog.load("mimic.hosp_pharmacy")()
p = p.assign(poe_id=p['poe_id'].map(lambda x: int(x[1])*10000 + int(x[0]) if x is not None else 0))

WARNING  Loading partitioned dataset as a whole, this may cause memory issues.                                           __init__.py:85


In [ ]:
p.to_parquet("../../data/pharma_tmp.pq")

In [ ]:
p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14736384 entries, 0 to 14736346
Data columns (total 27 columns):
 #   Column             Dtype         
---  ------             -----         
 0   subject_id         int32         
 1   hadm_id            int32         
 2   pharmacy_id        int32         
 3   poe_id             int64         
 4   starttime          datetime64[ns]
 5   stoptime           datetime64[ns]
 6   medication         category      
 7   proc_type          category      
 8   status             category      
 9   entertime          datetime64[ns]
 10  verifiedtime       datetime64[ns]
 11  route              category      
 12  frequency          category      
 13  disp_sched         category      
 14  infusion_type      category      
 15  sliding_scale      category      
 16  lockout_interval   category      
 17  basal_rate         category      
 18  one_hr_max         category      
 19  doses_per_24_hrs   category      
 20  duration           cat

In [ ]:
import pyarrow as pa

p_pa = pa.Table.from_pandas(p).drop(["__index_level_0__"])

In [ ]:
import pyarrow.csv as csv

with pa.CompressedOutputStream("../../data/pharma_tmp.csv.gz", "gzip") as out:
    csv.write_csv(p_pa, out)

In [ ]:
# $ gzip -dk pharma_tmp.csv.gz

In [ ]:
p_pq = pd.read_parquet("../../data/pharma_tmp.pq").reset_index(drop=True)
p_pq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14736384 entries, 0 to 14736383
Data columns (total 27 columns):
 #   Column             Dtype         
---  ------             -----         
 0   subject_id         int32         
 1   hadm_id            int32         
 2   pharmacy_id        int32         
 3   poe_id             int64         
 4   starttime          datetime64[ns]
 5   stoptime           datetime64[ns]
 6   medication         category      
 7   proc_type          category      
 8   status             category      
 9   entertime          datetime64[ns]
 10  verifiedtime       datetime64[ns]
 11  route              category      
 12  frequency          category      
 13  disp_sched         category      
 14  infusion_type      category      
 15  sliding_scale      category      
 16  lockout_interval   category      
 17  basal_rate         category      
 18  one_hr_max         category      
 19  doses_per_24_hrs   category      
 20  duration           cat

In [ ]:
%timeit pd.read_parquet("../../data/pharma_tmp.pq")

1.33 s ± 22.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
p_csv = pd.read_csv("../../data/pharma_tmp.csv")

WARNING  /tmp/ipykernel_2651520/949738715.py:1: DtypeWarning: Columns (16,18,24,26) have mixed types. Specify dtype     warnings.py:109
         option on import or set low_memory=False.                                                                                     
           p_csv = pd.read_csv("../../data/pharma_tmp.csv")                                                                            
                                                                                                                                       


In [ ]:
p_csv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14736384 entries, 0 to 14736383
Data columns (total 27 columns):
 #   Column             Dtype  
---  ------             -----  
 0   subject_id         int64  
 1   hadm_id            int64  
 2   pharmacy_id        int64  
 3   poe_id             int64  
 4   starttime          object 
 5   stoptime           object 
 6   medication         object 
 7   proc_type          object 
 8   status             object 
 9   entertime          object 
 10  verifiedtime       object 
 11  route              object 
 12  frequency          object 
 13  disp_sched         object 
 14  infusion_type      object 
 15  sliding_scale      object 
 16  lockout_interval   object 
 17  basal_rate         float64
 18  one_hr_max         object 
 19  doses_per_24_hrs   float64
 20  duration           float64
 21  duration_interval  object 
 22  expiration_value   float64
 23  expiration_unit    object 
 24  expirationdate     object 
 25  dispensation    

In [ ]:
p_csv = pd.read_csv("../../data/pharma_tmp.csv.gz")

WARNING  /tmp/ipykernel_2651520/2930756594.py:1: DtypeWarning: Columns (16,18,24,26) have mixed types. Specify dtype    warnings.py:109
         option on import or set low_memory=False.                                                                                     
           p_csv = pd.read_csv("../../data/pharma_tmp.csv.gz")                                                                         
                                                                                                                                       


In [ ]:
p_csv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14736384 entries, 0 to 14736383
Data columns (total 27 columns):
 #   Column             Dtype  
---  ------             -----  
 0   subject_id         int64  
 1   hadm_id            int64  
 2   pharmacy_id        int64  
 3   poe_id             int64  
 4   starttime          object 
 5   stoptime           object 
 6   medication         object 
 7   proc_type          object 
 8   status             object 
 9   entertime          object 
 10  verifiedtime       object 
 11  route              object 
 12  frequency          object 
 13  disp_sched         object 
 14  infusion_type      object 
 15  sliding_scale      object 
 16  lockout_interval   object 
 17  basal_rate         float64
 18  one_hr_max         object 
 19  doses_per_24_hrs   float64
 20  duration           float64
 21  duration_interval  object 
 22  expiration_value   float64
 23  expiration_unit    object 
 24  expirationdate     object 
 25  dispensation    

In [ ]:
p.dtypes


subject_id                    int32
hadm_id                       int32
pharmacy_id                   int32
poe_id                        int64
starttime            datetime64[ns]
stoptime             datetime64[ns]
medication                 category
proc_type                  category
status                     category
entertime            datetime64[ns]
verifiedtime         datetime64[ns]
route                      category
frequency                  category
disp_sched                 category
infusion_type              category
sliding_scale              category
lockout_interval           category
basal_rate                 category
one_hr_max                 category
doses_per_24_hrs           category
duration                   category
duration_interval          category
expiration_value           category
expiration_unit            category
expirationdate       datetime64[ns]
dispensation               category
fill_quantity              category
dtype: object


In [ ]:
p_csv = pd.read_csv(
    "../../data/pharma_tmp.csv",
    parse_dates=["expirationdate", "verifiedtime", "entertime", "stoptime", "starttime"],
    dtype={
        "subject_id": "int32",
        "hadm_id": "int32",
        "pharmacy_id": "int32",
        "poe_id": "int64",
        "medication": "category",
        "proc_type": "category",
        "status": "category",
        "route": "category",
        "frequency": "category",
        "disp_sched": "category",
        "infusion_type": "category",
        "sliding_scale": "category",
        "lockout_interval": "category",
        "basal_rate": "category",
        "one_hr_max": "category",
        "doses_per_24_hrs": "category",
        "duration": "category",
        "duration_interval": "category",
        "expiration_value": "category",
        "expiration_unit": "category",
        "dispensation": "category",
        "fill_quantity": "category",
    },
)


In [ ]:
p_csv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14736384 entries, 0 to 14736383
Data columns (total 27 columns):
 #   Column             Dtype         
---  ------             -----         
 0   subject_id         int32         
 1   hadm_id            int32         
 2   pharmacy_id        int32         
 3   poe_id             int64         
 4   starttime          datetime64[ns]
 5   stoptime           datetime64[ns]
 6   medication         category      
 7   proc_type          category      
 8   status             category      
 9   entertime          datetime64[ns]
 10  verifiedtime       datetime64[ns]
 11  route              category      
 12  frequency          category      
 13  disp_sched         category      
 14  infusion_type      category      
 15  sliding_scale      category      
 16  lockout_interval   category      
 17  basal_rate         category      
 18  one_hr_max         category      
 19  doses_per_24_hrs   category      
 20  duration           cat

In [ ]:
p_csv = pd.read_csv(
    "../../data/pharma_tmp.csv.gz",
    parse_dates=["expirationdate", "verifiedtime", "entertime", "stoptime", "starttime"],
    dtype={
        "subject_id": "int32",
        "hadm_id": "int32",
        "pharmacy_id": "int32",
        "poe_id": "int64",
        "medication": "category",
        "proc_type": "category",
        "status": "category",
        "route": "category",
        "frequency": "category",
        "disp_sched": "category",
        "infusion_type": "category",
        "sliding_scale": "category",
        "lockout_interval": "category",
        "basal_rate": "category",
        "one_hr_max": "category",
        "doses_per_24_hrs": "category",
        "duration": "category",
        "duration_interval": "category",
        "expiration_value": "category",
        "expiration_unit": "category",
        "dispensation": "category",
        "fill_quantity": "category",
    },
)


In [ ]:
p_csv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14736384 entries, 0 to 14736383
Data columns (total 27 columns):
 #   Column             Dtype         
---  ------             -----         
 0   subject_id         int32         
 1   hadm_id            int32         
 2   pharmacy_id        int32         
 3   poe_id             int64         
 4   starttime          datetime64[ns]
 5   stoptime           datetime64[ns]
 6   medication         category      
 7   proc_type          category      
 8   status             category      
 9   entertime          datetime64[ns]
 10  verifiedtime       datetime64[ns]
 11  route              category      
 12  frequency          category      
 13  disp_sched         category      
 14  infusion_type      category      
 15  sliding_scale      category      
 16  lockout_interval   category      
 17  basal_rate         category      
 18  one_hr_max         category      
 19  doses_per_24_hrs   category      
 20  duration           cat